# Snake-RL (Pytorch)
Gait Decomposition을 활용하여 Gait 자체를 학습하는 연구를 이 Jupyter 노트북에 작성을 하려고함. Gait Decomposition은 뱀 로봇의 움직임을 P함수와 M행렬로 분해하여 뱀 로봇의 Gait를 표현하는 방법이며, 이 방법을 통해서 직관적으로 뱀 로봇의 움직임을 표현할 수 있고, 파라미터 튜닝을 진행할 수 있다. 관련 내용은 [Arxiv 논문](https://arxiv.org/abs/2112.02057)을 참조할 것.

## Gait Decomposition에 대한 이해
Gait Decomposition에 대한 코드는 Gait 클래스에 구현되어 있다. 따라서 Gait 생성 코드를 코드에 import하자.

In [1]:
import numpy as np
from gait import gait

g = gait(1)

위 코드에서 삽입된 gait.py코드는 위 논문의 P함수와 M행렬을 객체로 구현한 코드이다.

Gait가 잘 생성되는지 확인해보자.

In [2]:
for i in range(0,10):
    print(g.generate(i).T)

[[15.  0. -0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0. -0.]]
[[ 0.   13.97 -0.   -0.    0.    0.   -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[  0.     0.   -23.31  -0.     0.     0.    -0.    -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[  0.     0.    -0.   -24.04   0.     0.    -0.    -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[ 0.    0.   -0.   -0.   28.53  0.   -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[ 0.    0.   -0.   -0.    0.   29.42 -0.   -0.    0.    0.   -0.   -0.
  -0.   -0.  ]]
[[  0.     0.    -0.    -0.     0.     0.   -29.96  -0.     0.     0.
   -0.    -0.    -0.    -0.  ]]
[[ -0.     0.    -0.    -0.     0.     0.    -0.   -29.08   0.     0.
   -0.    -0.     0.    -0.  ]]
[[-0.    0.   -0.   -0.    0.    0.   -0.   -0.   27.41  0.   -0.   -0.
   0.   -0.  ]]
[[-0.    0.   -0.   -0.    0.    0.   -0.   -0.    0.   23.07 -0.   -0.
   0.   -0.  ]]


위와 같이 생성된 Gait 데이터를 통해서 모터를 동작시키면 뱀 로봇이 움직이게 된다. Mujoco 시뮬레이터에서 움직이는 것을 그림으로 나타내면 아래 그림과 같다.

![fig1](./img/fig1.gif)

위 출력 결과와 같이 $i$가 증가하면서 각 모터에 입력되는 입력 신호가 바뀌는 것을 확인할 수 있다. 
위 결과는 P함수와 M행렬의 행렬 곱에서 대각 원소의 값을 출력한 결과이다. 위와 같은 결과를 모터에 전달하여 뱀 로봇이 움직이도록 만들 수 있다.

P함수는 모터의 목표 위치를 계산하는 함수로 주기 함수의 꼴을 갖는다. 주변 환경이 변하지 않는 이상적인 환경에서는 미리 학습된 P함수를 통해서 뱀 로봇을 움직이게 만들 수 있다.
하지만, 뱀 로봇이 항상 이상적인 환경에서 동작하지 않기 때문에 학습된 P함수가 최적 값을 내지못할 수도 있다. 우리는 이런 점에서 P함수를 센싱되는 지형의 데이터나 로봇의 상태에 따라서 변경할 수 있는 알고리즘을 구현하고자 한다.

이를 위해 P함수의 정의를 명확하게 하고자한다.

$$
P(t,A,\phi) \to \bar{\theta}
$$

위 식을 확인하면 P함수는 시간과 진폭, 위상차를 독립 변수로 갖는 함수이다. 더 상세하게 P함수를 나타내면 다음과 같다.

$$
\begin{equation*}
    \bar{\theta} = \left( 
    \begin{matrix}
    i = \text{odd}\;, \theta_i =  A_{\text{dor.}}\cdot \sin(\frac{2 \pi}{m}t + \frac{i}{2} \cdot \phi_{\text{dor.}})\;\;\; \\ \\
    i = \text{even}\;, \theta_i =   A_{\text{lat.}}\cdot \sin(\frac{2 \pi}{m}t + \frac{i-1}{2}\cdot \phi_{\text{lat.}})\; \end{matrix} 
    \right)\quad\quad
\end{equation*}
$$ 
여기에서, $m$은 P함수의 주파수 성분을 나타내기 위한 상수이며, $i$는 모터의 순서를 나타낸다.  우리 시스템에서는 Head 모터가 0번이다. $(i=0, \text{Head})$

이 때 다시 Gait 생성 코드를 보면 순서대로 머리모터부터 홀수 번째 모터들이 순서대로 움직이는 것으로 볼 수 있다. P함수는 t에 따라서 모든 모터에 대한 $\theta$값이 변경되는데 홀수번째 모터만 동작하는 이유는 우리가 뱀 로봇의 Gait를 정의하기 위해서 모션 행렬 M을 정의했기 때문이다. 모션 행렬 M은 뱀 로봇의 Gait를 구분하기 위한 주요한 파라미터이다. 이 행렬을 통해서 시간 t에서 어떤 모터가 동작할지 정할 수 있다. 현재 우리는 뱀 로봇의 Gait로 3가지 Gait를 정의했다. Inchworm, Serpentine, Sidewind이고 이를 모션 행렬로 표현하면 아래와 같다.

### Inchworm gait의 모션 행렬
$$
\begin{equation*}
    \mathbb{M_\text{inch}} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix} \end{equation*}
$$
### Serpentine gait의 모션 행렬
$$
\begin{equation}
    \mathbb{M_\text{ser}} = \begin{bmatrix}
1 & 0 & 0 & \cdots & 0 & 0\\
0 & 1 & 0 & \cdots & 0 & 0\\
0 & 0 & 1 & \cdots & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
 0 & 0 & 0 & \cdots & 1 & 0\\
 0 & 0 & 0 & \cdots & 0 & 1 
\end{bmatrix} = \mathbb{I}
\end{equation} 
$$
### Sidewind gait의 모션 행렬
$$
\begin{equation}
    \mathbb{M_\text{side}} = \begin{bmatrix}
0 & 1 & 0 & \cdots & 0 & 0 \\
1 & 0 & 0 & \cdots & 0 & 0 \\
0 & 0 & 0 & \cdots & 0 & 0 \\
0 & 0 & 1 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 0 & 1\\
0 & 0 & 0 & \cdots & 1 & 0\\
\end{bmatrix}
\end{equation}
$$

$\mathbb{m}$행렬은 k개의 열벡터로 이루어진다. $(\mathbb{M} = \{m_1,..m_k\})$ 여기에서 이 열벡터는 해당 시간에서 움직이는 모터의 인덱스를 표현한다. 따라서 P함수와 M행렬을 내적하면 어떤 모터가 어느 정도 움직이면 되는지를 표현할 수 있다. 여기에서 시스템 모터의 목표 각도 Array를 $\mathbb{G}$라고 가정하면, 수식으로 다음과 같이 표현할 수 있다.

$$
\begin{equation*}
\mathbb{M} = \left[
\begin{matrix}
    \vert & \vert &        & \vert \\
    m_1   &   m_2 & \cdots & m_k \\
    \vert & \vert &        & \vert \\
\end{matrix}
\right] \; , \, m_k \in \mathbb{R}^{14}\\
\end{equation*}
$$

$$
\begin{equation*}
    \mathbb{G}_k = \bar{\theta} \, \cdot \, {m_k}^{T}   \; , \; \mathbb{G}_{k} \in \mathbb{R}^{14 \times 14}
\end{equation*}
$$

이렇게 표현된 Gait는 P함수의 파라미터를 조절하여 튜닝(최적화)시킬 수 있다. 최적화를 진행하기 위해서는 Gait의 성능을 평가할 수 있는 범함수가 필요하다. 우리는 성능 평가를 위한 범함수를 아래와 같이 정의했다.

$$
\begin{equation}
    U(k,P(k,\,\bar{\theta}),\mathbb{M}) = {i} \cdot \Delta x - {j} \cdot |\Delta y| - {l} \cdot |\frac{\Delta y}{\Delta x}| - m \cdot {\int}_{t_{0}}^{t_f} {\| \mathbb{G}_{k} \|}_1 \,dk \\
    \\
\end{equation}
$$

위 범함수는 정의된 뱀 로봇이 10초 동안 움직이고 난 뒤의 결과를 통해 Gait 성능을 평가하는 함수이다. $i,j,l$은 성능 지표 이며 이 지표를 변경하면서 어떤 성능을 중점적으로 최적화할 것인지 표현할 수 있다. 위와 같은 범 함수를 활용하여 P함수의 파라미터를 최적화시킬 수 있었다. 하지만 이렇게 최적화된 Gait는 Open-loop로 제어되어, 뱀 로봇의 상태를 피드백받지 않고 모터의 목표값을 생성한다. 따라서 다양한 환경에서 적응하기 어렵다. 

우리는 이렇게 Decomposed된 Gait 구성 요소를 활용하여 Closed-loop제어를 하기위해서 DNN 네트워크를 활용하고자 한다. 

제안하는 네트워크 구조는 다음과 같다.





## State Listener 구현
위와 같은 구조로 뱀 로봇을 움직이기 위해서 시뮬레이션 결과를 받아올 수 있는 Listener를 구현해야 했다. 신경망 입력으로는 벡터 $\bar{x}$가 입력되며 이 벡터의 원소는 아래와 같다.

$$
\bar{x} = \left( \begin{align*} k \\ \theta \\ \dot{\theta} \\ \bar{q} \end{align*}\right)
$$

여기서 $k$는 time step을 나타내며, $\theta$는 관절의 각도, $\bar{q}$는 머리 링크의 오리엔테이션과 위치를 나타낸다.

이와 같은 데이터를 Mujoco 시뮬레이터에서 얻어야한다. Python에서 Mujoco를 import하고 관련된 설정을 진행한다.

In [3]:
import mujoco_py

snake = mujoco_py.load_model_from_path("../description/mujoco/snake_dgist.xml")
simulator = mujoco_py.MjSim(snake)

위와 같은 설정을 통해서 뱀 로봇 정의 파일을 Mujoco가 불러와 시뮬레이션 환경을 구축할 수 있게 되었다.

이후 각 Step마다 필요한 정보를 가져오는 함수를 구현하면 아래와 같다.

In [4]:
def stepListener(k) -> np.ndarray:
    joint_names = ['joint1','joint2','joint3','joint4','joint5','joint6','joint7','joint8','joint9','joint10','joint11','joint12','joint13','joint14']

    theta = np.array([simulator.data.get_joint_qpos(x) for x in joint_names])
    dtheta = np.array([simulator.data.get_joint_qvel(x) for x in joint_names])
    q = simulator.data.get_body_xquat('head')

    # print(theta)
    # print(dtheta)
    # print(q)

    return np.array([k, theta, dtheta, q],dtype='object')

이후 뱀 로봇이 P함수와 M행렬에 따라서 움직일 수 있도록 $\mathbb{G}$벡터를 계산하는 코드를 구현해야 한다. Gait Decomposition은 Gait 파라미터가 바뀌지 않는 상태에서 연속적으로 뱀 로봇의 움직임을 생성하는 반면에, 신경망을 통해 Feed forward하는 이 시스템의 경우 새롭게 Gait 파라미터를 받아 P함수의 특성을 바꾸는 기능이 추가되어야한다. 이를 코드로 구현하면 아래와 같다.

In [5]:
def genGait(k:int, gait:int, d_amp:float, d_phase:float, d_lam:float,l_amp:float, l_phase:float, l_lam:float, tau:int) -> np.ndarray:
    g.setParams(gait, d_amp, d_phase, d_lam, l_amp, l_phase, l_lam, tau)
    goal_P = g.generate(k)

    return goal_P

위와 같이 구현한 코드가 시뮬레이터에서 잘 작동하는지 확인해보도록 하자. 아직 신경망이 구현되지 않았으므로 임의의 Gait 파라미터를 생성하기 위해서 Random 패키지를 사용하여 Gait를 생성하기로 한다. 이를 코드로 구현하면 아래와 같다.

In [6]:
import random

viewer = mujoco_py.MjViewer(simulator) #For rendering sim results.

for k in range(0,5000):
    p1 = random.randint(0,900)/10
    p2 = random.randint(0,3600)/10
    p3 = random.randint(-10,10)
    p4 = random.randint(0,900)/10
    p5 = random.randint(0,3600)/10
    p6 = random.randint(-10,10)
    p7 = random.randint(1,4)

    G_k = genGait(k,1,p1,p2,p3,p4,p5,p6,p7)
    motor_idx = np.nonzero(G_k)

    for idx in motor_idx:
                if not(len(idx) == 0):
                    simulator.data.ctrl[idx] = g.degtorad(G_k[idx])

    simulator.step()
    viewer.render()


Creating window glfw


위 코드를 실행시키면 아래와 같은 모양으로 뱀 로봇이 움직이는 것을 확인할 수 있다.
![fig2](./img/randomized-movement.gif)

## Replay Memory 구현
Replay Memory는 학습 속도를 증가시키기 위해서 여러 케이스로 실험을 진행한 결과를 저장할 수 있도록 구현된 Memory이다. 일반적으로 학습에는 병렬 연산이 많이 이루어지므로 데이터를 모아 둔 뒤 한번에 학습하는 것이 더 빠르다고 한다. Replay Memory코드는 Pytorch 튜토리얼을 참고했다.



In [1]:
from collections import namedtuple, deque

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

NameError: name 'namedtuple' is not defined